# Question Answering with Sources

This notebook walks through how to use LangChain for question answering with sources over a list of documents. It covers three different chain types: `stuff`, `map_reduce`, and `refine`. For a more in depth explanation of what these chain types are, see TODO.

### Prepare Data
First we prepare the data. For this example we do similarity search over a vector database, but these documents could be fetched in any manner (the point of this notebook to highlight what to do AFTER you fetch the documents).

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document

In [2]:
with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [12]:
docsearch = FAISS.from_texts(texts, embeddings, metadatas=[{"source": i} for i in range(len(texts))])

In [13]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

In [3]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

### The `stuff` Chain

This sections shows results of using the `stuff` Chain to do question answering with sources.

In [4]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

In [4]:
docs = [Document(page_content=t, metadata={"source": i}) for i, t in enumerate(texts[:3])]

In [8]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president did not mention Justice Breyer.\nSOURCES: 0-pl, 1-pl, 2-pl'}

### The `map_reduce` Chain

This sections shows results of using the `map_reduce` Chain to do question answering with sources.

In [9]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_reduce")

In [10]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president did not mention Justice Breyer.\nSOURCES: 0, 1, 2'}

### The `refine` Chain

This sections shows results of using the `refine` Chain to do question answering with sources.

In [9]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine")

In [15]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "query_str": query}, return_only_outputs=True)

{'output_text': "\n\nThe president said that Justice Breyer has dedicated his life to serve the country and has left a legacy of excellence. He also thanked Justice Breyer for his service and for his commitment to advancing liberty and justice, including protecting the rights of women and the constitutional right affirmed in Roe v. Wade, preserving access to health care and a woman's right to choose, and advancing the bipartisan Equality Act to protect LGBTQ+ Americans. The president also noted that the State of the Union is strong because of the courage and determination of the American people, and that the nation will meet and overcome the challenges of our time as one people, just as the Ukrainian people have done in the face of adversity. Source: 0, 29, 35"}